In [33]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from abc import ABC, abstractmethod
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow import keras

In [34]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [35]:
class MnistClassifierInterface(ABC):
    @abstractmethod
    def train(self, x_train, y_train):
        pass

    @abstractmethod
    def predict(self, x_test):
        pass

In [36]:
class RandomForestMnist(MnistClassifierInterface):
    def __init__(self, n_estimators=100, max_depth=None, random_state=42, **kwargs):
         self.model = RandomForestClassifier(max_depth=np.int64(20), 
                                             min_samples_leaf=np.int64(1),
                                             min_samples_split=np.int64(4),
                                             n_estimators=np.int64(400),
                                             **kwargs)
        
    
    def train(self, x_train, y_train):
        x_train = x_train.reshape(x_train.shape[0], -1) 
        self.model.fit(x_train, y_train)
    
    def predict(self, x_test):
        x_test = x_test.reshape(x_test.shape[0], -1)
        return self.model.predict(x_test)

In [61]:
class FeedForwardMnist(MnistClassifierInterface):
    def __init__(self, optimizer="SGD", loss="categorical_crossentropy", metrics=["accuracy"]):
        self.model = keras.Sequential([
        keras.layers.Flatten(input_shape=[28,28]),    #model structure
        keras.layers.Dense(300, activation="relu"),
        keras.layers.Dense(100, activation="relu"),
        keras.layers.Dense(100, activation="relu"),
        keras.layers.Dense(10, activation="softmax")
        ])
        self.model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
        
    
    def train(self, x_train, y_train):
        x_train = x_train.astype("float32") / 255.0  #data normalization
        y_train = keras.utils.to_categorical(y_train, 10) #one-hot encoding
        self.model.fit(x_train, y_train, epochs=75, batch_size=64) #model training
    
    def predict(self, x_test):
        x_test = x_test.astype("float32") / 255.0  #data normalization
        predictions = self.model.predict(x_test)
        return predictions.argmax(axis=1)

In [70]:
class CnnMnist(MnistClassifierInterface):
    def __init__(self, optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]):
        self.model = keras.Sequential([
            keras.layers.Conv2D(64,7,activation='relu',padding="same",input_shape=[28,28,1]),
            keras.layers.MaxPooling2D(2),
            keras.layers.Conv2D(128,3,activation='relu',padding="same"),
            keras.layers.Conv2D(128,3,activation='relu',padding="same"),
            keras.layers.MaxPooling2D(2),
            keras.layers.Conv2D(128,3,activation='relu',padding="same"),
            keras.layers.Conv2D(128,3,activation='relu',padding="same"),
            keras.layers.MaxPooling2D(2),
            keras.layers.Flatten(),
            keras.layers.Dense(128,activation='relu'),
            keras.layers.Dropout(0.5),
            keras.layers.Dense(64,activation='relu'),
            keras.layers.Dropout(0.5),
            keras.layers.Dense(10, activation="softmax") 
        ])
        self.model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
        
    
    def train(self, x_train, y_train):
        x_train = x_train.astype("float32") / 255.0  #data normalization
        y_train = keras.utils.to_categorical(y_train, 10) #one-hot encoding
        self.model.fit(x_train, y_train, epochs=12, batch_size=64) #model training
    
    def predict(self, x_test):
        x_test = x_test.astype("float32") / 255.0  #data normalization
        predictions = self.model.predict(x_test)
        return predictions.argmax(axis=1)

In [71]:
class MnistClassifier:
    def __init__(self, algorithm):
        if algorithm == "rf":
            self.classifier = RandomForestMnist()
        elif algorithm == "nn":
            self.classifier = FeedForwardMnist()
        elif algorithm == "cnn":
            self.classifier = CnnMnist()
        else:
            raise ValueError('Unknown algorithm. Use "rf", "nn" or "cnn".')
    
    def train(self, x_train, y_train):
        self.classifier.train(x_train, y_train)
    
    def predict(self, x_test):
        return self.classifier.predict(x_test)


In [1]:
#RandomForest Test

In [46]:
rnd = MnistClassifier('rf')

In [47]:
rnd.train(x_train, y_train)

In [48]:
rnd_pred = rnd.predict(x_test)

In [49]:
rnd_pred

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

In [9]:
accuracy_rnd = accuracy_score(y_test, rnd_pred)
print(f"RandomForest accuracy on MNIST: {accuracy_rnd}")

RandomForest accuracy on MNIST: 0.97


In [2]:
#FeedForward Test

In [63]:
ffm = MnistClassifier('nn')

C:\inter\venv\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [64]:
ffm.train(x_train, y_train)

Epoch 1/75
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.5930 - loss: 1.4661
Epoch 2/75
938/938 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9020 - loss: 0.3500
Epoch 3/75
938/938 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9218 - loss: 0.2730
Epoch 4/75
938/938 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9329 - loss: 0.2327
Epoch 5/75
938/938 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9406 - loss: 0.2058
Epoch 6/75
938/938 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9485 - loss: 0.1815
Epoch 7/75
938/938 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9512 - loss: 0.1678
Epoch 8/75
938/938 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9567 - loss: 0.1473
Epoch 9/75
938/938 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9608 - loss: 0.1378
Epoch 10/75
938/938 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9647 - loss: 0.1251
Epoch 11/75
938/938 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9681 - loss: 0.1136
Epoch 12/75
938/938 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step

In [65]:
ffm_pred = ffm.predict(x_test) 

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step  


In [68]:
accuracy_ff = accuracy_score(y_test, ffm_pred)
print(f"FeedForward accuracy on MNIST: {accuracy_ff}")

FeedForward accuracy on MNIST: 0.9781


In [3]:
#CNN Test

In [77]:
cnn = MnistClassifier('cnn')

C:\inter\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [78]:
cnn.train(x_train, y_train)

Epoch 1/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 45s 45ms/step - accuracy: 0.7258 - loss: 0.8060
Epoch 2/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 42s 45ms/step - accuracy: 0.9649 - loss: 0.1406
Epoch 3/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 42s 45ms/step - accuracy: 0.9787 - loss: 0.0883
Epoch 4/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 42s 45ms/step - accuracy: 0.9857 - loss: 0.0669
Epoch 5/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 42s 45ms/step - accuracy: 0.9883 - loss: 0.0511
Epoch 6/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 41s 44ms/step - accuracy: 0.9877 - loss: 0.0501
Epoch 7/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 41s 44ms/step - accuracy: 0.9892 - loss: 0.0485
Epoch 8/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 41s 44ms/step - accuracy: 0.9927 - loss: 0.0309
Epoch 9/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 41s 44ms/step - accuracy: 0.9926 - loss: 0.0314
Epoch 10/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 41s 44ms/step - accuracy: 0.9929 - loss: 0.0309
Epoch 11/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 41s 44ms/step - accuracy: 0.9939 - loss: 0.0248
Epoch 12/12
938/938 ━━━━━━━━━━

In [79]:
cnn_pred = cnn.predict(x_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step


In [80]:
accuracy_cnn = accuracy_score(y_test, cnn_pred)
print(f"CNN on MNIST: {accuracy_cnn}")

FeedForward accuracy on MNIST: 0.993
